In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [12]:
# reflect an existing database into a new model
# reflect the tables
Base = automap_base()
Base.prepare(engine, reflect = True)



In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [18]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station
#first = session.query(Measurement.dsc()).first()
#first.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fb5376779d0>,
 'station': 'USC00519397',
 'date': '2010-01-01',
 'tobs': 65.0,
 'prcp': 0.08,
 'id': 1}

In [22]:
# Create our session (link) from Python to the DB
session = Session(engine)
engine.execute("select * from Measurement").fetchall()


[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0),
 (6, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (7, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (8, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (9, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (10, 'USC00519397', '2010-01-11', 0.01, 64.0),
 (11, 'USC00519397', '2010-01-12', 0.0, 61.0),
 (12, 'USC00519397', '2010-01-14', 0.0, 66.0),
 (13, 'USC00519397', '2010-01-15', 0.0, 65.0),
 (14, 'USC00519397', '2010-01-16', 0.0, 68.0),
 (15, 'USC00519397', '2010-01-17', 0.0, 64.0),
 (16, 'USC00519397', '2010-01-18', 0.0, 72.0),
 (17, 'USC00519397', '2010-01-19', 0.0, 66.0),
 (18, 'USC00519397', '2010-01-20', 0.0, 66.0),
 (19, 'USC00519397', '2010-01-21', 0.0, 69.0),
 (20, 'USC00519397', '2010-01-22', 0.0, 67.0),
 (21, 'USC00519397', '2010-01-23', 0.0, 67.0),
 (22, 'USC00519397

# Exploratory Climate Analysis

In [33]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
preps = session.query(Measurement.date,Measurement.prcp).order_by(Measurement.date.desc()).all()

for prep in preps:
    print(prep)
    
    #results= session.query(Emoji.emoji_char,Emoji.emoji_id,Emoji.score).order_by(Emoji.score.desc()).all()
#for result in results:
    #print(result)
#precipitation_query = session.query(Measurement).filter

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


('2017-08-23', 0.0)
('2017-08-23', 0.0)
('2017-08-23', 0.08)
('2017-08-23', 0.45)
('2017-08-22', 0.0)
('2017-08-22', 0.0)
('2017-08-22', 0.5)
('2017-08-21', 0.0)
('2017-08-21', 0.02)
('2017-08-21', None)
('2017-08-21', 0.56)
('2017-08-20', 0.0)
('2017-08-20', 0.01)
('2017-08-20', None)
('2017-08-19', 0.0)
('2017-08-19', 0.0)
('2017-08-19', None)
('2017-08-19', 0.09)
('2017-08-18', 0.0)
('2017-08-18', 0.0)
('2017-08-18', 0.06)
('2017-08-18', None)
('2017-08-17', 0.0)
('2017-08-17', 0.05)
('2017-08-17', 0.01)
('2017-08-17', 0.13)
('2017-08-16', 0.0)
('2017-08-16', 0.07)
('2017-08-16', 0.12)
('2017-08-16', 0.42)
('2017-08-15', 0.02)
('2017-08-15', 0.0)
('2017-08-15', 0.06)
('2017-08-15', 0.32)
('2017-08-15', 0.42)
('2017-08-14', 0.0)
('2017-08-14', 0.01)
('2017-08-14', 0.08)
('2017-08-14', 0.0)
('2017-08-14', 0.22)
('2017-08-13', 0.0)
('2017-08-13', None)
('2017-08-13', 0.0)
('2017-08-13', 0.0)
('2017-08-13', None)
('2017-08-12', 0.0)
('2017-08-12', 0.0)
('2017-08-12', 0.14)
('2017-08-11'

('2016-11-13', 0.0)
('2016-11-13', 0.0)
('2016-11-13', 0.0)
('2016-11-12', 0.0)
('2016-11-12', 0.0)
('2016-11-12', 0.0)
('2016-11-12', 0.0)
('2016-11-12', 0.0)
('2016-11-12', 0.0)
('2016-11-11', 0.0)
('2016-11-11', 0.0)
('2016-11-11', 0.0)
('2016-11-11', 0.0)
('2016-11-11', 0.0)
('2016-11-11', 0.0)
('2016-11-10', 0.0)
('2016-11-10', 0.0)
('2016-11-10', 0.0)
('2016-11-10', 0.0)
('2016-11-10', 0.0)
('2016-11-10', 0.01)
('2016-11-09', 0.0)
('2016-11-09', 0.08)
('2016-11-09', 0.17)
('2016-11-09', 0.0)
('2016-11-09', 0.0)
('2016-11-09', 0.11)
('2016-11-09', 0.04)
('2016-11-08', 0.07)
('2016-11-08', 0.14)
('2016-11-08', 0.02)
('2016-11-08', 0.15)
('2016-11-08', 0.21)
('2016-11-08', 0.53)
('2016-11-07', 0.0)
('2016-11-07', 0.0)
('2016-11-07', 0.13)
('2016-11-07', None)
('2016-11-07', 0.0)
('2016-11-07', 0.0)
('2016-11-07', 0.05)
('2016-11-06', 0.0)
('2016-11-06', 0.02)
('2016-11-06', 0.0)
('2016-11-06', 0.0)
('2016-11-06', 0.01)
('2016-11-06', 0.05)
('2016-11-05', 0.0)
('2016-11-05', 0.02)
('

('2016-03-21', 0.06)
('2016-03-21', 0.0)
('2016-03-21', None)
('2016-03-21', 0.0)
('2016-03-21', 0.0)
('2016-03-20', 0.0)
('2016-03-20', 0.0)
('2016-03-20', 0.0)
('2016-03-20', 0.0)
('2016-03-20', 0.0)
('2016-03-19', 0.0)
('2016-03-19', 0.02)
('2016-03-19', 0.03)
('2016-03-19', 0.16)
('2016-03-19', 0.22)
('2016-03-18', 0.0)
('2016-03-18', 0.03)
('2016-03-18', 0.13)
('2016-03-18', 0.0)
('2016-03-18', 0.08)
('2016-03-18', 0.07)
('2016-03-17', 0.0)
('2016-03-17', 0.02)
('2016-03-17', 0.0)
('2016-03-17', 0.0)
('2016-03-17', 0.01)
('2016-03-17', 0.0)
('2016-03-16', 0.05)
('2016-03-16', 0.41)
('2016-03-16', 0.06)
('2016-03-16', 0.0)
('2016-03-16', None)
('2016-03-16', 0.23)
('2016-03-16', 0.37)
('2016-03-15', 0.08)
('2016-03-15', 0.3)
('2016-03-15', 1.08)
('2016-03-15', None)
('2016-03-15', 0.82)
('2016-03-15', 0.57)
('2016-03-14', 0.0)
('2016-03-14', 0.0)
('2016-03-14', None)
('2016-03-14', None)
('2016-03-14', None)
('2016-03-14', 0.03)
('2016-03-14', 0.16)
('2016-03-13', 0.0)
('2016-03-13

('2015-05-30', 0.14)
('2015-05-30', 0.01)
('2015-05-30', 0.01)
('2015-05-30', 0.01)
('2015-05-29', 0.0)
('2015-05-29', 0.01)
('2015-05-29', 0.0)
('2015-05-29', 0.0)
('2015-05-29', 0.04)
('2015-05-29', 0.0)
('2015-05-29', 0.0)
('2015-05-28', 0.0)
('2015-05-28', 1.29)
('2015-05-28', 0.64)
('2015-05-28', 0.0)
('2015-05-28', 0.68)
('2015-05-28', 0.72)
('2015-05-28', 0.0)
('2015-05-28', 0.03)
('2015-05-27', 0.0)
('2015-05-27', 0.07)
('2015-05-27', 0.05)
('2015-05-27', 0.13)
('2015-05-27', 0.36)
('2015-05-27', 0.0)
('2015-05-27', 0.03)
('2015-05-26', 0.0)
('2015-05-26', 0.08)
('2015-05-26', 0.06)
('2015-05-26', 0.0)
('2015-05-26', 0.0)
('2015-05-26', 0.07)
('2015-05-26', None)
('2015-05-26', 0.04)
('2015-05-25', 0.0)
('2015-05-25', 0.01)
('2015-05-25', 0.0)
('2015-05-25', 0.0)
('2015-05-25', 0.0)
('2015-05-25', 0.0)
('2015-05-24', 0.0)
('2015-05-24', 0.0)
('2015-05-24', 0.0)
('2015-05-24', 0.0)
('2015-05-24', 0.0)
('2015-05-24', 0.0)
('2015-05-23', 0.07)
('2015-05-23', 0.8)
('2015-05-23', 1.

('2014-10-11', 0.07)
('2014-10-11', 0.0)
('2014-10-11', 0.16)
('2014-10-10', 0.0)
('2014-10-10', 0.08)
('2014-10-10', 0.01)
('2014-10-10', None)
('2014-10-10', None)
('2014-10-10', 0.13)
('2014-10-10', 0.0)
('2014-10-10', 0.1)
('2014-10-09', 0.0)
('2014-10-09', 0.01)
('2014-10-09', 0.0)
('2014-10-09', 0.06)
('2014-10-09', 0.0)
('2014-10-09', None)
('2014-10-08', 0.0)
('2014-10-08', 0.13)
('2014-10-08', 0.0)
('2014-10-08', 0.04)
('2014-10-08', 0.15)
('2014-10-08', 0.0)
('2014-10-07', 1.22)
('2014-10-07', 0.0)
('2014-10-07', 0.07)
('2014-10-07', 0.0)
('2014-10-07', 0.03)
('2014-10-07', 0.24)
('2014-10-07', 0.06)
('2014-10-06', 0.24)
('2014-10-06', 0.26)
('2014-10-06', None)
('2014-10-06', 0.2)
('2014-10-06', 0.1)
('2014-10-06', 0.38)
('2014-10-06', 0.58)
('2014-10-05', 0.0)
('2014-10-05', 0.64)
('2014-10-05', None)
('2014-10-05', 0.26)
('2014-10-05', 0.95)
('2014-10-05', 0.0)
('2014-10-05', 0.27)
('2014-10-04', 0.0)
('2014-10-04', 0.33)
('2014-10-04', 0.0)
('2014-10-04', None)
('2014-10-

...

('2013-04-06', 0.17)
('2013-04-06', 0.0)
('2013-04-05', 0.0)
('2013-04-05', 0.0)
('2013-04-05', 0.0)
('2013-04-05', 0.0)
('2013-04-05', 0.0)
('2013-04-05', 0.0)
('2013-04-05', 0.0)
('2013-04-05', 0.0)
('2013-04-04', 0.0)
('2013-04-04', 0.07)
('2013-04-04', 0.0)
('2013-04-04', 0.02)
('2013-04-04', 0.02)
('2013-04-04', 0.11)
('2013-04-04', 0.0)
('2013-04-04', 0.36)
('2013-04-03', 0.0)
('2013-04-03', 0.02)
('2013-04-03', 0.0)
('2013-04-03', 0.0)
('2013-04-03', 0.05)
('2013-04-03', 0.0)
('2013-04-03', 0.0)
('2013-04-03', 0.0)
('2013-04-03', 0.0)
('2013-04-02', 0.0)
('2013-04-02', 0.0)
('2013-04-02', 0.0)
('2013-04-02', 0.0)
('2013-04-02', 0.01)
('2013-04-02', 0.02)
('2013-04-02', 0.0)
('2013-04-02', 0.0)
('2013-04-01', 0.0)
('2013-04-01', 0.0)
('2013-04-01', None)
('2013-04-01', 0.0)
('2013-04-01', 0.0)
('2013-04-01', 0.0)
('2013-04-01', 0.0)
('2013-04-01', None)
('2013-03-31', 0.25)
('2013-03-31', 0.02)
('2013-03-31', 0.02)
('2013-03-31', 0.04)
('2013-03-31', 0.0)
('2013-03-30', 0.01)
('2

('2012-06-29', 0.0)
('2012-06-29', None)
('2012-06-29', 0.06)
('2012-06-29', 0.14)
('2012-06-29', 0.0)
('2012-06-29', 0.79)
('2012-06-28', 0.0)
('2012-06-28', 0.0)
('2012-06-28', 0.0)
('2012-06-28', 0.0)
('2012-06-28', 0.0)
('2012-06-28', 0.01)
('2012-06-28', 0.0)
('2012-06-28', 0.37)
('2012-06-27', 0.01)
('2012-06-27', 0.23)
('2012-06-27', 0.2)
('2012-06-27', None)
('2012-06-27', 0.15)
('2012-06-27', 0.43)
('2012-06-27', 0.01)
('2012-06-27', 0.73)
('2012-06-26', 0.15)
('2012-06-26', 0.22)
('2012-06-26', 0.03)
('2012-06-26', None)
('2012-06-26', None)
('2012-06-26', 0.19)
('2012-06-26', 0.22)
('2012-06-26', 0.0)
('2012-06-26', 0.92)
('2012-06-25', 0.03)
('2012-06-25', 0.04)
('2012-06-25', 0.12)
('2012-06-25', None)
('2012-06-25', 0.51)
('2012-06-25', 0.13)
('2012-06-25', 0.01)
('2012-06-25', 0.89)
('2012-06-24', 0.0)
('2012-06-24', 0.27)
('2012-06-24', None)
('2012-06-24', 0.0)
('2012-06-24', 0.14)
('2012-06-24', 0.0)
('2012-06-24', 1.13)
('2012-06-23', 0.0)
('2012-06-23', 0.28)
('2012

('2011-10-01', 0.0)
('2011-10-01', 0.0)
('2011-09-30', 0.11)
('2011-09-30', 0.02)
('2011-09-30', 0.0)
('2011-09-30', 0.0)
('2011-09-30', 0.08)
('2011-09-30', 0.0)
('2011-09-30', 0.0)
('2011-09-29', 0.02)
('2011-09-29', 0.0)
('2011-09-29', 0.0)
('2011-09-29', 0.0)
('2011-09-29', 0.03)
('2011-09-29', 0.0)
('2011-09-29', 0.09)
('2011-09-28', 0.0)
('2011-09-28', 0.03)
('2011-09-28', 0.02)
('2011-09-28', 0.0)
('2011-09-28', 0.04)
('2011-09-28', 0.1)
('2011-09-28', 0.01)
('2011-09-28', 0.1)
('2011-09-27', 0.0)
('2011-09-27', 0.0)
('2011-09-27', 0.0)
('2011-09-27', 0.0)
('2011-09-27', 0.0)
('2011-09-27', 0.0)
('2011-09-27', 0.0)
('2011-09-27', 0.0)
('2011-09-26', 0.0)
('2011-09-26', 0.0)
('2011-09-26', 0.0)
('2011-09-26', 0.0)
('2011-09-26', None)
('2011-09-26', 0.0)
('2011-09-26', 0.0)
('2011-09-26', 0.0)
('2011-09-26', 0.0)
('2011-09-25', 0.0)
('2011-09-25', 0.0)
('2011-09-25', 0.0)
('2011-09-25', 0.0)
('2011-09-25', 0.0)
('2011-09-24', 0.0)
('2011-09-24', 0.0)
('2011-09-24', 0.0)
('2011-09

('2011-01-09', 0.0)
('2011-01-09', 0.0)
('2011-01-08', 0.0)
('2011-01-08', 0.01)
('2011-01-08', 0.0)
('2011-01-08', 0.0)
('2011-01-08', 0.01)
('2011-01-08', 0.0)
('2011-01-08', 0.0)
('2011-01-07', 0.0)
('2011-01-07', 0.02)
('2011-01-07', 0.0)
('2011-01-07', 0.02)
('2011-01-07', 0.0)
('2011-01-07', 0.02)
('2011-01-07', 0.0)
('2011-01-07', 0.0)
('2011-01-07', 0.0)
('2011-01-06', 0.01)
('2011-01-06', 0.21)
('2011-01-06', 0.13)
('2011-01-06', 0.0)
('2011-01-06', 0.14)
('2011-01-06', 0.15)
('2011-01-06', 0.16)
('2011-01-06', 0.22)
('2011-01-05', 0.0)
('2011-01-05', 0.04)
('2011-01-05', 0.0)
('2011-01-05', 0.0)
('2011-01-05', 0.15)
('2011-01-05', 0.0)
('2011-01-05', 0.08)
('2011-01-05', 0.0)
('2011-01-05', 0.09)
('2011-01-04', 0.0)
('2011-01-04', 0.0)
('2011-01-04', 0.0)
('2011-01-04', 0.0)
('2011-01-04', None)
('2011-01-04', 0.0)
('2011-01-04', 0.0)
('2011-01-04', 0.0)
('2011-01-04', 0.05)
('2011-01-03', 0.0)
('2011-01-03', 0.01)
('2011-01-03', 0.04)
('2011-01-03', 0.0)
('2011-01-03', 0.07)

('2010-05-03', 0.14)
('2010-05-02', 0.03)
('2010-05-02', 0.03)
('2010-05-02', 0.02)
('2010-05-02', 0.0)
('2010-05-02', 0.12)
('2010-05-02', 0.11)
('2010-05-02', 0.15)
('2010-05-02', 0.37)
('2010-05-01', 0.0)
('2010-05-01', 0.02)
('2010-05-01', 0.24)
('2010-05-01', 0.29)
('2010-05-01', 0.0)
('2010-05-01', 0.39)
('2010-05-01', 0.0)
('2010-05-01', 0.31)
('2010-04-30', 0.0)
('2010-04-30', 0.02)
('2010-04-30', 0.02)
('2010-04-30', None)
('2010-04-30', 0.0)
('2010-04-30', 0.17)
('2010-04-30', 0.04)
('2010-04-30', 0.92)
('2010-04-29', 0.01)
('2010-04-29', 0.01)
('2010-04-29', 0.03)
('2010-04-29', 0.07)
('2010-04-29', 0.08)
('2010-04-29', 1.29)
('2010-04-28', 0.0)
('2010-04-28', 0.07)
('2010-04-28', 0.09)
('2010-04-28', None)
('2010-04-28', 0.03)
('2010-04-28', 0.04)
('2010-04-28', 0.0)
('2010-04-28', 0.49)
('2010-04-27', 0.01)
('2010-04-27', 0.02)
('2010-04-27', 0.0)
('2010-04-27', 0.0)
('2010-04-27', 0.0)
('2010-04-27', 0.0)
('2010-04-26', 0.0)
('2010-04-26', 0.0)
('2010-04-26', 0.0)
('2010-

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
